In [1]:
import pandas
import random
import json

# Extract relevant data from SNN11 dataset

In [2]:
demographic_data_nl = pandas.read_csv("ESS11-NL-subset/ESS11-subset.csv")
# df_noord_holland_urban = demographic_data_nl[(demographic_data_nl['region'] == 'NL33') & (demographic_data_nl['domicil'] == 1) | (demographic_data_nl['domicil'] == 2)] #urban areas in Noord Holland
# df_NL_urban = demographic_data_nl[(demographic_data_nl['domicil'] == 1) | (demographic_data_nl['domicil'] == 2)] #Urban (and suburban/urban-adjacent) areas in the netherlands

pandas.set_option('display.max_columns', None)


In [ ]:
chosen_cols = ['region', #Region
               'domicil' #urbanicity
               'pspwght',
               'idno', #id
               'agea', #age
                'gndr', #sex
                'nobingnd', #gender
                'cntbrthd', 'brncntr', 'ctzcntr', 'facntr', 'fbrncntc', 'mocntr', 'mbrncntc', 'feethngr', 'anctrya1', 'anctrya2',#ethnicity and nationality
                'rlgblg', 'rlgdnanl', #religion; NB: replace with 'rlgblg' for generalized religion
                'edlvenl', #education; NB: replace with 'edulvlb' for generalized education
                'pdwrk', 'edctn', 'uempla', 'uempli', 'dsbld', 'rtrd', 'cmsrv', 'hswrk', 'dngoth', #main activity (employment)
                'fltdpr', 'flteeff', 'slprl', 'wrhpp', 'fltlnl', 'enjlf', 'fltsd', 'cldgng', #well-being: proxies for depreeesion (PHQ-9)
                ]

rename_dict = {
    'region': 'Region',
    'domicil': 'Urbanicity',
    'pspwght': 'Weight',
    'idno': 'ParticipantID',
    'agea': 'Age',
    'gndr': 'SexAtBirth',
    'nobingnd': 'GenderIdentitySelfReported',
    'cntbrthd': 'CountryOfBirth', 'brncntr': 'BornInNL', 'ctzcntr': 'CitizenOfNL', 'facntr':'FatherBornInNL', 'fbrncntc': 'FatherCountryOfBirth', 'mocntr':'MotherBornInNL', 'mbrncntc': 'MotherCountryOfBirth', 'anctrya1':'ancestry1', 'anctrya2':'ancestry2',#nationality
    'feethngr': 'FeelsSameEthnicGroup',
    'rlgblg': 'ReligiousBelonging',
    'rlgdnanl': 'ReligiousDenominationNL',
    'edlvenl': 'EducationLevelNL',
    'pdwrk': 'PaidWork', 'edctn': 'InEducation', 'uempla': 'UnemployedActive', 'uempli': 'UnemployedInactive', 'dsbld': 'PermanentlySick', 'rtrd': 'Retired', 'cmsrv': 'CommunityService', 'hswrk': 'HouseworkChildcare', 'dngoth': 'Other', #main activity (employment)
    'fltdpr': 'FeltDepressed', 'flteeff': 'LowEnergy', 'slprl': 'SleepProblems', 'wrhpp': 'FeltHappy', 'fltlnl': 'FeltLonely', 'enjlf': 'EnjoyedLife', 'fltsd': 'FeltSad', 'cldgng': 'TroubleGettingGoing'
}


In [4]:
df = demographic_data_nl[chosen_cols]
df = df.rename(columns=rename_dict)

print(df.head())


  Region    Weight  ParticipantID  Age  SexAtBirth  \
0   NL12  0.813390          50072   36           1   
1   NL33  0.768768          50144   48           1   
2   NL32  0.574372          50166   59           1   
3   NL11  1.081618          50193   46           2   
4   NL22  0.934341          50202   80           2   

   GenderIdentitySelfReported CountryOfBirth  BornInNL  CitizenOfNL  \
0                           1           6666         1            1   
1                           1           6666         1            1   
2                           1             IR         2            1   
3                           2           6666         1            1   
4                           2           6666         1            1   

   FatherBornInNL FatherCountryOfBirth  MotherBornInNL MotherCountryOfBirth  \
0               1                 6666               1                 6666   
1               1                 6666               1                 6666   
2          

# Translate the dataset into our cohort questionnaire data structure

Region (only for synthesizing, will be removed later!)

In [5]:
with open("region_mapping_nl.json", "r", encoding="utf-8") as f:
    region_map = json.load(f)

df["Region"] = df["Region"].map(region_map)

Categorize age

In [6]:
def translate_age(age):
    if age < 18:
        return 'Onder de 18 jaar'
    elif 18 <= age <= 24:
        return '18-24 jaar'
    elif 25 <= age <= 34:
        return '25-34 jaar'
    elif 35 <= age <= 44:
        return '35-44 jaar'
    elif 45 <= age <= 54:
        return '45-54 jaar'
    elif 55 <= age <= 64:
        return '55-64 jaar'
    else:
        return '65 jaar of ouder'

df['AgeGroup'] = df['Age'].apply(translate_age)

Categorize sex

In [7]:
df['SexAtBirth'] = df['SexAtBirth'].map({1: 'Man', 2: 'Vrouw'})

Categorize gender identity

In [8]:
def translate_gender_identity(row): #TODO: check if this corresponds to the questionnaire
    if row['GenderIdentitySelfReported'] == 1 and row['SexAtBirth'] == 'Man':
        return 'Man'
    elif row['GenderIdentitySelfReported'] == 2 and row['SexAtBirth'] == 'Vrouw':
        return 'Vrouw'
    elif row['GenderIdentitySelfReported'] == 2 and row['SexAtBirth'] == 'Man':
        return 'Trans vrouw'
    elif row['GenderIdentitySelfReported'] == 1 and row['SexAtBirth'] == 'Vrouw':
        return 'Trans man'
    elif row['GenderIdentitySelfReported'] == 3:
        return random.choice(['Genderqueer', 'Gender non-binair', 'Anders (specificeer)']) #if 'other' make a random choice between the three
    else:
        return 'Ik wil het liever niet zeggen'
    
df['GenderIdentity'] = df.apply(translate_gender_identity, axis=1)


Education mapping

In [9]:
education_mapping_snn11 = {
    1: 'Basisonderwijs',
    2: 'Basisonderwijs',
    3: 'Vmbo/Mavo',
    4: 'Vmbo/Mavo',
    5: 'Mbo (niveau 1-4)',
    6: 'Havo/Vwo',
    7: 'Havo/Vwo',
    8: 'Mbo (niveau 1-4)',
    9: 'Mbo (niveau 1-4)',
    10: 'Mbo (niveau 1-4)',
    11: 'Mbo (niveau 1-4)',
    12: 'HBO',
    13: 'HBO',
    14: 'HBO',
    15: 'Universiteit (Bachelor, Master, of hoger)',
    16: 'Universiteit (Bachelor, Master, of hoger)',
    17: 'Universiteit (Bachelor, Master, of hoger)',
    18: 'Universiteit (Bachelor, Master, of hoger)',
    5555: 'Anders (specificeer)',
    7777: 'Ik wil het liever niet zeggen',
    8888: 'Ik wil het liever niet zeggen',
    9999: 'Ik wil het liever niet zeggen'
}

df['EducationLevelNL'] = df['EducationLevelNL'].map(education_mapping_snn11).fillna('N/A')

Employment mapping

In [10]:
activity_mapping = {
    'PaidWork': 'Betaalde baan (fulltime)', #note that part-time isn't marked
    'InEducation': 'Scholier of student',
    'UnemployedActive': 'Geen betaald werk om andere redenen',
    'UnemployedInactive': 'Geen betaald werk om andere redenen',
    'PermanentlySick': 'Gedeeltelijk / geen betaald werk vanwege gezondheidsproblemen',
    'Retired': 'Gepensioneerd of met prepensioen',
    'CommunityService': 'Vrijwilligerswerk',
    'HouseworkChildcare': 'Zorg voor het huishouden (eventueel kinderen)',
    'Other': 'Anders (specificeer)'
}


def determine_work_status(row):
    for var, label in activity_mapping.items():
        if row[var] == 1:
            return label
    return 'Ik wil het liever niet zeggen'

df['WorkStatus'] = df.apply(determine_work_status, axis=1)


Religious mapping

In [11]:
religion_mapping_snn11 = {
    1: 'Christendom',
    2: 'Christendom',
    3: 'Christendom',
    4: 'Christendom',
    5: 'Christendom',
    6: 'Christendom',
    7: 'Christendom',
    8: 'Christendom',
    9: 'Christendom',
    10: 'Christendom',
    11: 'Christendom',
    12: 'Christendom',
    13: 'Christendom',
    14: 'Christendom',
    15: 'Christendom',
    16: 'Christendom',
    17: 'Christendom',
    18: 'Christendom',
    19: 'Christendom',
    20: 'Christendom',
    21: 'Christendom',
    22: 'Hindoeïsme',
    23: 'Boeddhisme',
    24: 'Anders (specificeer)',
    25: 'Jodendom',
    26: 'Islam',
    27: 'Anders (specificeer): Humanistisch Verbond',
    28: 'Anders (specificeer): Andere niet-christelijke religies',
    6666: 'Geen religie',
    7777: 'Ik wil het liever niet zeggen',
    9999: 'Ik wil het liever niet zeggen'
}


def map_religion(row):
    if row['ReligiousBelonging'] in [7, 8, 9]:
        return 'Ik wil het liever niet zeggen'
    elif row['ReligiousBelonging'] == 2:
        return 'Geen religie'
    elif row['ReligiousBelonging'] == 1:
        return religion_mapping_snn11.get(row['ReligiousDenominationNL'])
    else: 
        return 'Anders (specificeer): ' + str(row['ReligiousDenominationNL'])



df['Religion'] = df.apply(map_religion, axis=1)


Ethnicity and nationality variables

In [12]:
with open("country_mapping_dutch.json", "r", encoding="utf-8") as f:
    country_mapping = json.load(f)

#country of birth
df['CountryOfBirth'] = [
    'Nederlands' if born == 1 else country_mapping.get(str(code), code)
    for born, code in zip(df['BornInNL'], df['CountryOfBirth'])
]

#mother's country of birth   
df['MotherCountryOfBirth'] = [
    'Nederlands' if born == 1 else country_mapping.get(str(code), code)
    for born, code in zip(df['MotherBornInNL'], df['MotherCountryOfBirth'])
]
    
# Father's Country of Birth
df['FatherCountryOfBirth'] = [
    'Nederlands' if born == 1 else country_mapping.get(str(code), code)
    for born, code in zip(df['FatherBornInNL'], df['FatherCountryOfBirth'])
]
# Nationality
df['Nationality'] = [
    'Nederlands' if citizen == 1 else country_mapping.get(str(code), code)
    for citizen, code in zip(df['CitizenOfNL'], df['CountryOfBirth'])
]

#Ethnicity
with open('ancestry_mapping_snn11.json', 'r') as f:
    ancestry_mapping = json.load(f)
        
df['Ethnicity'] = df['ancestry1'].map(lambda x: ancestry_mapping.get(str(x), 'Anders (specificeer)'))


In [13]:
#display all rows
# pandas.set_option('display.max_rows', None)

#ethnicity check
# df[['FeelsSameEthnicGroup', 'CountryOfBirth', 'MotherCountryOfBirth', 'FatherCountryOfBirth','Nationality', 'Ethnicity', 'Religion']] #TODO: look if data is clean. Now country of birth values are not; check religion

Extract relevant data for the cohort

In [14]:
extracted_data  = df[['Region',
                    'Weight', 
                    'ParticipantID',
                     'AgeGroup',
                     'SexAtBirth',
                     'GenderIdentity',
                     'CountryOfBirth', 'Nationality', 'MotherCountryOfBirth', 'FatherCountryOfBirth', 'Ethnicity',
                     'Religion',
                     'EducationLevelNL',
                     'WorkStatus',
                     'FeltDepressed', 'LowEnergy', 'SleepProblems', 'FeltHappy', 'FeltLonely', 'EnjoyedLife', 'FeltSad', 'TroubleGettingGoing',]]


extracted_data.to_csv('extracted_data.csv')                   

In [15]:
extracted_data.head()

,Region,Weight,ParticipantID,AgeGroup,SexAtBirth,GenderIdentity,CountryOfBirth,Nationality,MotherCountryOfBirth,FatherCountryOfBirth,Ethnicity,Religion,EducationLevelNL,WorkStatus,FeltDepressed,LowEnergy,SleepProblems,FeltHappy,FeltLonely,EnjoyedLife,FeltSad,TroubleGettingGoing
0,Friesland (NL),0.813390,50072,35-44 jaar,Man,Man,Nederlands,Nederlands,Nederlands,Nederlands,Nederlands,Geen religie,HBO,Betaalde baan (fulltime),1,1,1,4,1,4,1,1
1,Zuid-Holland,0.768768,50144,45-54 jaar,Man,Man,Nederlands,Nederlands,Nederlands,Nederlands,Nederlands,Geen religie,HBO,Betaalde baan (fulltime),1,2,1,3,1,4,2,1
2,Noord-Holland,0.574372,50166,55-64 jaar,Man,Man,"Anders: Iran, Islamic Republiek van",Nederlands,"Anders: Iran, Islamic Republiek van","Anders: Iran, Islamic Republiek van",Anders (specificeer): Iranian,Geen religie,"Universiteit (Bachelor, Master, of hoger)",Betaalde baan (fulltime),1,1,1,3,1,3,1,1
3,Groningen,1.081618,50193,45-54 jaar,Vrouw,Vrouw,Nederlands,Nederlands,Nederlands,Nederlands,Nederlands,Geen religie,Mbo (niveau 1-4),Betaalde baan (fulltime),1,1,1,4,1,3,1,2
4,Gelderland,0.934341,50202,65 jaar of ouder,Vrouw,Vrouw,Nederlands,Nederlands,Nederlands,Nederlands,Nederlands,Geen religie,Vmbo/Mavo,Gepensioneerd of met prepensioen,1,4,1,2,1,4,2,4


In [16]:
#breakdown of WorkStatus values
work_summary = df['WorkStatus'].value_counts(dropna=False).to_frame(name='Count')
work_summary['Percentage'] = (work_summary['Count'] / work_summary['Count'].sum() * 100).round(2)
print(work_summary)

                                                    Count  Percentage
WorkStatus                                                           
Betaalde baan (fulltime)                             1077       63.54
Gepensioneerd of met prepensioen                      368       21.71
Gedeeltelijk / geen betaald werk vanwege gezond...     80        4.72
Scholier of student                                    69        4.07
Zorg voor het huishouden (eventueel kinderen)          47        2.77
Geen betaald werk om andere redenen                    34        2.01
Anders (specificeer)                                   16        0.94
Ik wil het liever niet zeggen                           4        0.24


In [17]:
# ppl who don't identify with their sex at birth
df[df['SexAtBirth'] != df['GenderIdentity']]
# print(len(df[df['SexAtBirth'] != df['GenderIdentity']]))

,Region,Weight,ParticipantID,Age,SexAtBirth,GenderIdentitySelfReported,CountryOfBirth,BornInNL,CitizenOfNL,FatherBornInNL,FatherCountryOfBirth,MotherBornInNL,MotherCountryOfBirth,FeelsSameEthnicGroup,ancestry1,ancestry2,ReligiousBelonging,ReligiousDenominationNL,EducationLevelNL,PaidWork,InEducation,UnemployedActive,UnemployedInactive,PermanentlySick,Retired,CommunityService,HouseworkChildcare,Other,FeltDepressed,LowEnergy,SleepProblems,FeltHappy,FeltLonely,EnjoyedLife,FeltSad,TroubleGettingGoing,AgeGroup,GenderIdentity,WorkStatus,Religion,Nationality,Ethnicity
25,Zuid-Holland,1.475254,50671,21,Man,3,Nederlands,1,1,1,Nederlands,1,Nederlands,1,11040,555555,2,6666,Mbo (niveau 1-4),0,0,0,1,0,0,0,0,0,2,3,3,3,2,3,2,3,18-24 jaar,Gender non-binair,Geen betaald werk om andere redenen,Geen religie,Nederlands,Nederlands
47,Friesland (NL),1.100245,51064,59,Vrouw,7,Nederlands,1,1,1,Nederlands,1,Nederlands,1,11060,11040,1,18,Mbo (niveau 1-4),1,0,0,0,0,0,0,0,0,1,1,1,3,1,3,1,2,55-64 jaar,Ik wil het liever niet zeggen,Betaalde baan (fulltime),Christendom,Nederlands,Nederlands
332,Drenthe,1.163608,57148,22,Vrouw,3,Nederlands,1,1,1,Nederlands,1,Nederlands,1,11040,555555,2,6666,Havo/Vwo,1,1,0,0,0,0,0,0,0,2,2,2,2,2,2,3,2,18-24 jaar,Genderqueer,Betaalde baan (fulltime),Geen religie,Nederlands,Nederlands
407,Gelderland,0.806730,58840,43,Vrouw,3,Nederlands,1,1,1,Nederlands,1,Nederlands,1,11040,11070,2,6666,HBO,1,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,1,35-44 jaar,Gender non-binair,Betaalde baan (fulltime),Geen religie,Nederlands,Nederlands
536,Noord-Holland,0.899569,61543,58,Vrouw,7,Anders: Germany,2,1,2,Anders: Germany,2,Anders: Germany,1,11070,11040,2,6666,Basisonderwijs,0,0,0,0,0,0,0,0,1,3,3,3,2,2,3,3,3,55-64 jaar,Ik wil het liever niet zeggen,Anders (specificeer),Geen religie,Nederlands,Anders (specificeer): German nfs
549,Noord-Holland,1.231693,61839,20,Vrouw,3,Nederlands,1,1,1,Nederlands,2,Anders: Verenigde Koninkrijk,1,11040,11030,2,6666,Havo/Vwo,1,1,0,0,0,0,0,0,0,2,2,3,2,1,2,2,2,18-24 jaar,Anders (specificeer),Betaalde baan (fulltime),Geen religie,Nederlands,Nederlands
588,Noord-Holland,1.309164,62696,69,Man,3,Nederlands,1,1,1,Nederlands,1,Nederlands,1,11040,555555,2,6666,Mbo (niveau 1-4),0,0,0,0,0,1,0,0,0,2,2,1,3,2,2,2,1,65 jaar of ouder,Anders (specificeer),Gepensioneerd of met prepensioen,Geen religie,Nederlands,Nederlands
1046,Noord-Brabant,1.027970,73094,49,Vrouw,3,Anders: Yugoslavia,2,2,2,Anders: Yugoslavia,2,Anders: Yugoslavia,1,14160,555555,2,6666,Havo/Vwo,1,0,0,0,0,0,0,0,0,1,1,2,4,1,4,1,1,45-54 jaar,Anders (specificeer),Betaalde baan (fulltime),Geen religie,Anders: Yugoslavia,Anders (specificeer): Yugoslav
1495,Noord-Brabant,1.407455,82269,21,Man,3,Nederlands,1,1,1,Nederlands,1,Nederlands,1,11040,555555,2,6666,Vmbo/Mavo,1,0,0,0,0,0,0,1,0,2,2,2,3,2,2,2,1,18-24 jaar,Gender non-binair,Betaalde baan (fulltime),Geen religie,Nederlands,Nederlands


Resampling attempt

In [18]:
import numpy as np

rng = np.random.default_rng(42)
p = extracted_data['Weight'] / extracted_data['Weight'].sum()
print(p[30:50])
k = min(round(extracted_data['Weight'].sum()), len(extracted_data))  # max n unique rows

sel_idx = rng.choice(extracted_data.index, size=k, replace=False, p=p)
df_weighted_unique = extracted_data.loc[sel_idx].reset_index(drop=True)


30    0.000339
31    0.000544
32    0.000785
33    0.000446
34    0.000655
35    0.000451
36    0.000881
37    0.000606
38    0.000344
39    0.000672
40    0.000408
41    0.000649
42    0.000341
43    0.000513
44    0.000689
45    0.000630
46    0.000766
47    0.000649
48    0.000633
49    0.000513
Name: Weight, dtype: float64


In [19]:
print(len(df_weighted_unique))

1695
